# **Similarity between two images**

## **Import**

In [7]:
!pip install keras
!pip install tensorflow
!pip install numpy
!pip install matplotlib
!pip install sklearn

  Using cached sklearn-0.0-py2.py3-none-any.whl


In [1]:
# Import libraries and modules
from keras.datasets import cifar100
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns

## **Dataset**

### Load Dataset

In [3]:
# load dataset

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Split x_train in half and make x_validation
x_validation = x_test[0:5000]
x_test = x_test[5000:]
y_validation = y_test[0:5000]
y_test = y_test[5000:]

assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (5000, 32, 32, 3)
assert x_validation.shape == (5000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_validation.shape == (5000, 1)
assert y_test.shape == (5000, 1)

print('x_train shape:', x_train.shape)
print('x_validation shape:', x_validation.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_validation shape:', y_validation.shape)
print('y_test shape:', y_test.shape)

x_train shape: (50000, 32, 32, 3)
x_validation shape: (5000, 32, 32, 3)
x_test shape: (5000, 32, 32, 3)
y_train shape: (50000, 1)
y_validation shape: (5000, 1)
y_test shape: (5000, 1)


In [4]:
classes = {
    0: "apple",
    1: "aquarium_fish",
    2: "baby",
    3: "bear",
    4: "beaver",
    5: "bed",
    6: "bee",
    7: "beetle",
    8: "bicycle",
    9: "bottle",
    10: "bowl",
    11: "boy",
    12: "bridge",
    13: "bus",
    14: "butterfly",
    15: "camel",
    16: "can",
    17: "castle",
    18: "caterpillar",
    19: "cattle",
    20: "chair",
    21: "chimpanzee",
    22: "clock",
    23: "cloud",
    24: "cockroach",
    25: "couch",
    26: "cra",
    27: "crocodile",
    28: "cup",
    29: "dinosaur",
    30: "dolphin",
    31: "elephant",
    32: "flatfish",
    33: "forest",
    34: "fox",
    35: "girl",
    36: "hamster",
    37: "house",
    38: "kangaroo",
    39: "keyboard",
    40: "lamp",
    41: "lawn_mower",
    42: "leopard",
    43: "lion",
    44: "lizard",
    45: "lobster",
    46: "man",
    47: "maple_tree",
    48: "motorcycle",
    49: "mountain",
    50: "mouse",
    51: "mushroom",
    52: "oak_tree",
    53: "orange",
    54: "orchid",
    55: "otter",
    56: "palm_tree",
    57: "pear",
    58: "pickup_truck",
    59: "pine_tree",
    60: "plain",
    61: "plate",
    62: "poppy",
    63: "porcupine",
    64: "possum",
    65: "rabbit",
    66: "raccoon",
    67: "ray",
    68: "road",
    69: "rocket",
    70: "rose",
    71: "sea",
    72: "seal",
    73: "shark",
    74: "shrew",
    75: "skunk",
    76: "skyscraper",
    77: "snail",
    78: "snake",
    79: "spider",
    80: "squirrel",
    81: "streetcar",
    82: "sunflower",
    83: "sweet_pepper",
    84: "table",
    85: "tank",
    86: "telephone",
    87: "television",
    88: "tiger",
    89: "tractor",
    90: "train",
    91: "trout",
    92: "tulip",
    93: "turtle",
    94: "wardrobe",
    95: "whale",
    96: "willow_tree",
    97: "wolf",
    98: "woman",
    99: "worm"
}

### Preprocess Dataset

In [12]:
# preprocess dataset



# scale pixels
def prep_pixels(x):
	# convert from integers to floats
	x_norm = x.astype('float32')
	# normalize to range 0-1
	x_norm = x_norm / 255.0
	# return normalized images
	return x_norm

y_train = to_categorical(y_train, num_classes=classes.__len__())
y_validation = to_categorical(y_validation, num_classes=classes.__len__())
y_test = to_categorical(y_test, num_classes=classes.__len__())

x_train = prep_pixels(x_train)
x_validation = prep_pixels(x_validation)
x_test = prep_pixels(x_test)

x_train[0].shape

(32, 32, 3)

## **Model**

### Define and Train Model

In [7]:
# Build model

def build_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(classes.__len__(), activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = build_model()

epochs = 10
batch_size = 128

model.summary()

history = model.fit(x_train, y_train, validation_data=(x_validation, y_validation), epochs=epochs, batch_size=batch_size)

# save model
model.save('./model/similarity_model.h5')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 conv2d_7 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 15, 15, 32)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 15, 15, 64)        18496     
                                                                 
 conv2d_9 (Conv2D)           (None, 13, 13, 64)        36928     
                                                      

### Export as tensorflow.js model

In [11]:
# Export model as tensorflow.js model

!tensorflowjs_converter --input_format=keras ./model/similarity_model.h5 ./model/tfjs


### Evaluate Model

In [8]:
_, acc = model.evaluate(x_test, y_test, verbose=0)
print('%.3f' % (acc))

0.326


## **Test**

### Plot diagnostic learning curves

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	fig, ax = plt.subplots(1, 2, figsize=(10, 5))
	ax[0].set_title('Cross Entropy Loss')
	ax[0].set_xlabel('Epoch')
	ax[0].set_ylabel('Loss')
	ax[0].plot(history.history['loss'], color='blue', label='train')
	ax[0].plot(history.history['val_loss'], color='orange', label='validation')
	ax[0].legend()

	# plot accuracy
	ax[1].set_title('Classification Accuracy')
	ax[1].set_xlabel('Epoch')
	ax[1].set_ylabel('Accuracy')
	ax[1].plot(history.history['accuracy'], color='blue', label='train')
	ax[1].plot(history.history['val_accuracy'], color='orange', label='validation')
	ax[1].legend()
	
	plt.show()

summarize_diagnostics(history)

### Data outside of the dataset

### Kappa Score

In [ ]:
def calculate_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

pred = model.predict(x_test)
pred = np.argmax(pred, axis=1)

kappa = calculate_kappa(old_y_test, pred)
print("Kappa: ", kappa)

### Confusion Matrix

In [ ]:
# sns confusion matrix

cm = confusion_matrix(old_y_test, pred)

cm = cm / cm.astype(float).sum(axis=1) # This line normalizes the calculated confusion matrix

figure = plt.figure(figsize=(8, 8))
sns.heatmap(cm, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

## **Convert to tflite**

In [ ]:
model=tf.keras.models.load_model('./model/similarity_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)